In [8]:
import zipfile
import csv
import pandas as pd
import os
import logging
import mysql.connector
from datetime import datetime


In [9]:
# A connection to the database table established only when the user provides the proper target host, port and user credentials. 
def get_db_connection():
    try: 
        connection = mysql.connector.connect(user='root',
                                            password = 'password',
                                            host = 'localhost',
                                            port ='3306',
                                            database = 'datapipelinemini')
    except Exception as error:
        print("Error while connecting to database for job tracker: ", error)
    return connection
connection = get_db_connection()

In [10]:
def create_table(connection):
    cursor = connection.cursor()
    schema = [
        'ticket_id INT',
        'trans_date DATE',
        'event_id INT',
        'event_name VARCHAR(50)',
        'event_date DATE',
        'event_type VARCHAR(10)',
        'event_city VARCHAR(20)',
        'customer_id INT',
        'price DECIMAL',
        'num_tickets INT']
    statement = "CREATE TABLE ticket_sales ({},{},{},{},{},{},{},{},{},{})".format(*schema)
    cursor.execute(statement)
    connection.commit()
    cursor.close()
create_table(connection)

ProgrammingError: 1050 (42S01): Table 'ticket_sales' already exists

In [5]:
# Insert each record of the CSV File into the sales table
def load_third_party(connection, file_path_csv='third_party_sales_1 (1).csv'):
    cursor = connection.cursor()
    sql = 'INSERT INTO ticket_sales (ticket_id,'
    sql = sql + 'trans_date,event_id,event_name,event_date,event_type,'
    sql = sql + 'event_city,customer_id,price,num_tickets) VALUES '
    sql = sql + '(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
    
    with open(file_path_csv, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            row[1] = datetime.strptime(row[1], '%Y-%m-%d')
            cursor.execute(sql, row)
        
    # [Iterate through the CSV file and execute insert statement]
    # cursor.execute('INSERT INTO logs Values ( a,b,c)', args)
    
    
    connection.commit()
    cursor.close()
    return

load_third_party(connection)

In [6]:
# Get top-selling tickets for past month
def query_popular_tickets(connection):
     # Get the most popular ticket in the past month
    sql_statement = "select event_name, num_tickets from ticket_sales order by num_tickets desc limit 3"
    cursor = connection.cursor()
    cursor.execute(sql_statement)
    records = cursor.fetchall()
    return records

[('Washington Spirits vs Sky Blue FC', 5),
 ('The North American International Auto Show', 3),
 ('Christmas Spectacular', 3)]

In [11]:
def print_popular(data):
    print("Here are the most popular tickets in the past month:")
    for row in data:
        print("{}: {}".format(row[0], row[1]))
print_popular(query_popular_tickets(connection))

Here are the most popular tickets in the past month:
Washington Spirits vs Sky Blue FC: 5
The North American International Auto Show: 3
Christmas Spectacular: 3
